<a href="https://colab.research.google.com/github/gc2321/LangChain-Python/blob/main/1_Parsing_Output.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import os

In [1]:
!pip install --upgrade --quiet langchain-openai langchain-community
!pip install -quiet openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.2/310.2 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.7/320.7 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 973.5/973.5 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.3/124.3 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.9 MB/s eta 0:00:00

Usage:   
  pip3 install [options

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:
open_ai_api ="/content/gdrive/MyDrive/llm/api_key.txt"
f = open(open_ai_api)
api_key = f.read()

In [6]:
from langchain.prompts import PromptTemplate, SystemMessagePromptTemplate,ChatPromptTemplate, HumanMessagePromptTemplate
from langchain_openai import OpenAI
from langchain.chat_models import ChatOpenAI

model = ChatOpenAI(openai_api_key=api_key)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


## List Parsing

In [7]:
from langchain.output_parsers import CommaSeparatedListOutputParser

In [8]:
output_parser = CommaSeparatedListOutputParser()

In [9]:
format_instructions = output_parser.get_format_instructions()

In [10]:
print(format_instructions)

Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`


In [11]:
reply = "one, two, three"
output_parser.parse("one, two, three")

['one', 'two', 'three']

In [12]:
human_template = '{request} {format_instructions}'
human_prompt = HumanMessagePromptTemplate.from_template(human_template)

In [13]:
chat_prompt = ChatPromptTemplate.from_messages([human_prompt])

chat_prompt.format_prompt(request="give me 5 characteristics of dogs",
                   format_instructions = output_parser.get_format_instructions())

ChatPromptValue(messages=[HumanMessage(content='give me 5 characteristics of dogs Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`')])

In [14]:
request = chat_prompt.format_prompt(request="give me 5 characteristics of dogs",
                   format_instructions = output_parser.get_format_instructions()).to_messages()

In [15]:
result = model.invoke(request)

In [16]:
result.content

'Loyal, playful, protective, friendly, intelligent'

In [17]:
# Convert to desired output:
output_parser.parse(result.content)

['Loyal', 'playful', 'protective', 'friendly', 'intelligent']

## Datetime Parse

In [18]:
from langchain.output_parsers import DatetimeOutputParser

In [19]:
output_parser = DatetimeOutputParser()

In [20]:
print(output_parser.get_format_instructions())

Write a datetime string that matches the following pattern: '%Y-%m-%dT%H:%M:%S.%fZ'.

Examples: 371-08-22T22:59:20.308689Z, 732-03-25T11:58:33.038208Z, 620-04-11T20:42:35.771351Z

Return ONLY this string, no other words!


In [21]:
template_text = "{request}\n{format_instructions}"
human_prompt=HumanMessagePromptTemplate.from_template(template_text)

In [22]:
print(chat_prompt.format(request="When was the 13th Amendment ratified in the US?",
                   format_instructions=output_parser.get_format_instructions()
                   ))

Human: When was the 13th Amendment ratified in the US? Write a datetime string that matches the following pattern: '%Y-%m-%dT%H:%M:%S.%fZ'.

Examples: 1739-09-20T23:30:33.119453Z, 377-01-02T01:04:23.318155Z, 2013-11-30T21:52:30.103432Z

Return ONLY this string, no other words!


In [23]:
request = chat_prompt.format_prompt(request="What date was the 13th Amendment ratified in the US?",
                   format_instructions=output_parser.get_format_instructions()
                   ).to_messages()

In [24]:
result = model.invoke(request,temperature=0)

In [25]:
result.content

'1865-12-06T00:00:00.000000Z'

In [26]:
output_parser.parse(result.content)

datetime.datetime(1865, 12, 6, 0, 0)

## Methods To Fix Parsing Issues

#### Auto-Fix Parse

In [27]:
from langchain.output_parsers import OutputFixingParser

output_parser = DatetimeOutputParser()

misformatted = result.content

In [28]:
misformatted

'1865-12-06T00:00:00.000000Z'

In [29]:
new_parser = OutputFixingParser.from_llm(parser=output_parser, llm=model)

In [30]:
new_parser.parse(misformatted)

datetime.datetime(1865, 12, 6, 0, 0)

### Fixing via System Prompt

In [31]:
system_prompt = SystemMessagePromptTemplate.from_template("You always reply to questions only in datetime patterns.")
template_text = "{request}\n{format_instructions}"
human_prompt=HumanMessagePromptTemplate.from_template(template_text)

In [32]:
chat_prompt = ChatPromptTemplate.from_messages([system_prompt,human_prompt])

In [33]:
print(chat_prompt.format(request="When was the 13th Amendment ratified in the US?",
                   format_instructions=output_parser.get_format_instructions()
                   ))

System: You always reply to questions only in datetime patterns.
Human: When was the 13th Amendment ratified in the US?
Write a datetime string that matches the following pattern: '%Y-%m-%dT%H:%M:%S.%fZ'.

Examples: 1762-08-17T17:02:13.484184Z, 894-01-25T05:01:39.034138Z, 1812-07-19T02:53:19.835938Z

Return ONLY this string, no other words!


In [34]:
request = chat_prompt.format_prompt(request="What date was the 13th Amendment ratified in the US?",
                   format_instructions=output_parser.get_format_instructions()
                   ).to_messages()

In [35]:
result = model.invoke(request,temperature=0)

In [36]:
result.content

'1865-12-06T00:00:00.000000Z'

In [37]:
output_parser.parse(result.content)

datetime.datetime(1865, 12, 6, 0, 0)

## Pydantic JSON Parser

In [38]:
from langchain.output_parsers import PydanticOutputParser

In [39]:
from pydantic import BaseModel, Field

In [40]:
class Scientist(BaseModel):
    name: str = Field(description="Name of a Scientist")
    discoveries: list = Field(description="Python list of discoveries")

In [41]:
query = 'Name a famous scientist and a list of their discoveries'

In [42]:
parser = PydanticOutputParser(pydantic_object=Scientist)

In [43]:
print(parser.get_format_instructions())

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"name": {"description": "Name of a Scientist", "title": "Name", "type": "string"}, "discoveries": {"description": "Python list of discoveries", "items": {}, "title": "Discoveries", "type": "array"}}, "required": ["name", "discoveries"]}
```


In [44]:
prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

_input = prompt.format_prompt(query="Tell me about a famous scientist")

output = model.invoke(_input.to_string())

parser.parse(output.content)

Scientist(name='Albert Einstein', discoveries=['Theory of General Relativity', 'Photoelectric Effect', 'Mass-energy equivalence (E=mc^2)'])